# Импорт

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image as Image
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from LookGenerator.datasets.classes import PersonSegmentationDataset
from LookGenerator.networks.segmentation import UNet, train_unet

# Загрузка данных

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [3]:
batch_size=1

In [8]:
train_dataset = PersonSegmentationDataset("C:\\Users\\asus\\Desktop\\datasets\\VITONHD\\train", transform=transform, segmentation_type="image-densepose")
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
print(len(train_dataset))

11647


In [9]:
val_dataset = PersonSegmentationDataset("C:\\Users\\asus\\Desktop\\datasets\\VITONHD\\test", transform=transform, segmentation_type="image-densepose")
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
len(val_dataloader)

2032

In [10]:
data = None
targets = None
for X, y in train_dataloader:
    print(X.shape)
    print(y.shape)
    data = X
    targets = y
    break

KeyError: 'image-densepose'

# Обучение модели

Метки классов на изображении:

1. Левое лицо                       248,251,14
2. Правое лицо                      251,235,25
3. Туловище                         20,80,194
4. Внешняя верхняя левая рука       190,189,96
5. Внешняя верхняя правая рука      215,187,88
6. Внешняя нижняя левая рука        252,206,48
7. Внешняя нижняя правая рука       250,220,36
8. Внутренняя верхняя левая рука    145,191,116
9. Внутренняя верхняя правая рука   170,190,105
10. Внетренняя нижняя левая рука     228,191,74
11. Внутренняя нижняя правая рука    240,198,60
12. Левая кисть                      8,110,221
13. Правая кисть                     4,98,224
14. Левая нога                       22,173,184
15. Правая нога                      6,166,198

In [ ]:
model = UNet(in_channels=3, out_channels = 3)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
device = 'gpu' if torch.cuda.is_available() else 'cpu'

In [ ]:
output = model(data)
criterion = torch.nn.CrossEntropyLoss()
loss = criterion(output, targets)
print(loss.item())

In [ ]:
train_history, val_history = train_unet(
    model,
    train_dataloader,
    val_dataloader,
    optimizer,
    device=device,
    epoch_num=5
)